In [2]:
import json
import pandas as pd

In [12]:
df = pd.read_json("./500_pairs.json")

'''
    Get average token count for both target and source.
'''

def tokenize(text):
    return text.split(" ")

# print(df["source"][0])

tkn_len_src = 0
tkn_len_trg = 0

for source_report in df["source"]:
    tkn_len_src += len(tokenize(source_report))
    
for target_report in df["target"]:
    tkn_len_trg += len(tokenize(target_report))
    
tkn_len_src /= 500
tkn_len_trg /= 500

print(tkn_len_src)
print(tkn_len_trg)

total_reports = 500



348.192
209.778


In [18]:
# Price menu: 
pricing = {
    "gpt-4o (api)": {"input":2.50, "output": 10},
    "gpt-4o-mini (api)": {"input":0.15, "output": 0.6},
    # "deepseek-chat (api)": {"input":0.135, "output": 0.55},   # discount hour, cache miss
    "deepseek-chat (api)": {"input":0.27, "output": 1.10},   # non-discount hour, cache miss
    "t5-small (local)": {"input":0, "output": 0},
    "bart-large-cnn (local)":  {"input":0, "output": 0},
    "MedLlama-3-8B-v2.0 (local)": {"input":0, "output":0}
}

# https://platform.openai.com/docs/pricing
# https://api-docs.deepseek.com/quick_start/pricing
# https://huggingface.co/johnsnowlabs/JSL-MedLlama-3-8B-v2.0

In [19]:
# Selected Models
lexical_models = {
    "L2": "gpt-4o (api)",
    "L3": "deepseek-chat (api)",
    "L4": "MedLlama-3-8B-v2.0 (local)"
}
syntactic_models = {
    "S1": "gpt-4o (api)",
    "S3": "deepseek-chat (api)",
    "S4": "t5-small (local)",
    "S5": "bart-large-cnn (local)",
    "S6": "MedLlama-3-8B-v2.0 (local)"
}
format_models = {
    "F1": "gpt-4o-mini (api)",
    "F2": "t5-small (local)",
    "F3": "bart-large-cnn (local)",
    "F4": "deepseek-chat (api)",
    "F5": "MedLlama-3-8B-v2.0 (local)"
}

# Token stats (replace with real averages)
avg_input_tokens = tkn_len_src
avg_output_tokens = tkn_len_trg
N = 500  # reports

# Fan-out call multipliers
calls_per_model = {
    "Lexical": N,
    "Syntactic": N * len(lexical_models),
    "Format": N * len(lexical_models) * len(syntactic_models)
}

# Models grouped by layer
models_by_layer = {
    "Lexical": lexical_models,
    "Syntactic": syntactic_models,
    "Format": format_models
}

# Calculate cost
print("=== FINAL COST ESTIMATION (Layered Tree Execution) ===\n")

for layer, models in models_by_layer.items():
    print(f"--- {layer} Layer ---")
    call_count = calls_per_model[layer]
    
    for model_id, model_name in models.items():
        cost_in = (avg_input_tokens * call_count / 1_000_000) * pricing[model_name]["input"]
        cost_out = (avg_output_tokens * call_count / 1_000_000) * pricing[model_name]["output"]
        total_cost = cost_in + cost_out
        
        print(f"{model_name.ljust(25)} | Calls: {call_count:<5} | Cost: ${total_cost:.4f}")
    print()


=== FINAL COST ESTIMATION (Layered Tree Execution) ===

--- Lexical Layer ---
gpt-4o (api)              | Calls: 500   | Cost: $1.4841
deepseek-chat (api)       | Calls: 500   | Cost: $0.1624
MedLlama-3-8B-v2.0 (local) | Calls: 500   | Cost: $0.0000

--- Syntactic Layer ---
gpt-4o (api)              | Calls: 1500  | Cost: $4.4524
deepseek-chat (api)       | Calls: 1500  | Cost: $0.4872
t5-small (local)          | Calls: 1500  | Cost: $0.0000
bart-large-cnn (local)    | Calls: 1500  | Cost: $0.0000
MedLlama-3-8B-v2.0 (local) | Calls: 1500  | Cost: $0.0000

--- Format Layer ---
gpt-4o-mini (api)         | Calls: 7500  | Cost: $1.3357
t5-small (local)          | Calls: 7500  | Cost: $0.0000
bart-large-cnn (local)    | Calls: 7500  | Cost: $0.0000
deepseek-chat (api)       | Calls: 7500  | Cost: $2.4358
MedLlama-3-8B-v2.0 (local) | Calls: 7500  | Cost: $0.0000



In [21]:
total_cost = 1.4841 + 0.1624 + 4.4524 + 0.4872 + 1.3357 + 2.4358
total_cost

10.3576